<a href="https://colab.research.google.com/github/MonaJB/F2Informatica/blob/main/progetto_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fitz
!pip install PyMuPDF
!pip install Unidecode
!pip install Pillow
!apt-get install -y tesseract-ocr
!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import fitz
from unidecode import unidecode
from tabulate import tabulate
import re
import pytesseract
from PIL import Image
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def mostra_df(df):
    # Imposta il numero massimo di righe e colonne visualizzate
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.max_columns', None)

    # Mostra il DataFrame
    print(df)

In [ ]:
def check_if_fincons(doc):
  pagina = doc[0]
  # Converti la pagina in un'immagine
  immagine = pagina.get_pixmap()
  # Salva l'immagine in un file temporaneo
  nome_file_temporaneo = "temp.png"
  Image.frombytes("RGB", [immagine.width, immagine.height], immagine.samples).save(nome_file_temporaneo)

  # Usa pytesseract per estrarre il testo dall'immagine
  testo_estratto = pytesseract.image_to_string(Image.open(nome_file_temporaneo))

  return 'fincons' in testo_estratto[:20].lower()

In [ ]:
def extract_pdf_to_json_folder(folder_path, output_file):
    # Create an empty dictionary to store data for all PDFs
    all_data = {}

    # Iterate over each file in the specified folder
    for file_name in os.listdir(folder_path):
        # Check if the file has a PDF extension
        if file_name.lower().endswith('.pdf'):
            # Construct the full path to the PDF file
            pdf_path = os.path.join(folder_path, file_name)

            # Extract data from the PDF and append it to the dictionary
            pdf_data = extract_pdf_to_df(pdf_path)
            all_data[file_name] = pdf_data

    # Save the data to the output file with each key-value pair on a single line
    with open(output_file, 'w') as f:
        for key, value in all_data.items():
            f.write(f'"{key}": {json.dumps(value, indent=None)}\n')

    # Return the combined data for all PDFs in JSON format
    return json.dumps(all_data, indent=2)

In [ ]:
def extract_pdf_folder(folder_path):
    # Create an empty dictionary to store data for all PDFs
    all_data = {}

    # Iterate over each file in the specified folder
    for file_name in os.listdir(folder_path):
        # Check if the file has a PDF extension
        if file_name.lower().endswith('.pdf'):
            # Construct the full path to the PDF file
            pdf_path = os.path.join(folder_path, file_name)

            # Extract data from the PDF and append it to the dictionary
            pdf_data = extract_pdf_to_df(pdf_path)
            all_data[file_name] = pdf_data

    return all_data

In [ ]:
def extract_pdf_to_df(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text()

    ## controlla se il file è di Fincons
    if check_if_fincons(doc):

        pattern = r'Dettaglio RDA n\. (\d+) /'

        # separate particular text blocks and unicode them
        output = []

        for page in doc:
            output += page.get_text("blocks")

        previous_block_id = 0  # Set a variable to mark the block id

        for block in output:
            if block[6] == 0:  # We only take the text
                #if previous_block_id != block[5]:  # Compare the block number
                    #print("\n")
                plain_text = unidecode(block[4])
                #print(plain_text)

        # store all of these texts in a DataFrame
        # The x0, y0, x1, y1 is the coordinate of the text line in the document.
        df = pd.DataFrame(output, columns=['x0', 'y0', 'x1', 'y1', 'text', 'block_no', 'block_type'])
        # print(df.head())

        df['text'] = df['text'].apply(lambda x: unidecode(x))

        df = df.drop(df[df.block_type == 1].index)

        # analyze the text
        # get the spans from the PDF file
        block_dict = {}

        page_num = 1

        for page in doc:  # Iterate all pages in the document
            file_dict = page.get_text('dict')  # Get the page dictionary
            block = file_dict['blocks']  # Get the block information
            block_dict[page_num] = block  # Store in block dictionary
            page_num += 1  # Increase the page value by 1

        # retrieve the spans and store them in a DataFrame
        import re

        spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
        rows = []

        for page_num, blocks in block_dict.items():
            for block in blocks:
                if block['type'] == 0:
                    for line in block['lines']:
                        for span in line['spans']:
                            xmin, ymin, xmax, ymax = list(span['bbox'])
                            font_size = span['size']
                            text = unidecode(span['text'])
                            span_font = span['font']

                            is_upper = False
                            is_bold = False

                            if "bold" in span_font.lower():
                                is_bold = True

                            if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                                is_upper = True

                            if text.replace(" ", "") != "":
                                rows.append((xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size))

        span_df = pd.DataFrame(rows, columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'is_upper', 'is_bold', 'span_font',
                                              'font_size'])

        #  finding the tag for the text
        span_scores = []
        span_num_occur = {}
        special = '[(_:/,#%\=@)]'

        for index, span_row in span_df.iterrows():
            score = round(span_row.font_size)
            text = span_row.text

            if not re.search(special, text):
                if span_row.is_bold:
                    score += 1

                if span_row.is_upper:
                    score += 1

            span_scores.append(score)

        values, counts = np.unique(span_scores, return_counts=True)

        # the numer of unique text styles in the document, and the number of its occurrences
        style_dict = {}

        for value, count in zip(values, counts):
            style_dict[value] = count

        sorted(style_dict.items(), key=lambda x: x[1])

        p_size = max(style_dict, key=style_dict.get)
        idx = 0
        tag = {}

        for size in sorted(values, reverse=True):
            idx += 1
            if size == p_size:
                idx = 0
                tag[size] = 'p'

            if size > p_size:
                tag[size] = 'h{0}'.format(idx)

            if size < p_size:
                tag[size] = 's{0}'.format(idx)
        # add tag column
        span_tags = [tag[score] for score in span_scores]
        span_df['tag'] = span_tags
        # create a new dataframe where and store the text by headings
        headings_list = []
        text_list = []
        tmp = []
        # print(span_df)
        # out = pd.DataFrame(span_df)
        heading = ''
        lista_keys_due_righe = ["""pc o altro)"""]
        for index, span_row in span_df.iterrows():
            text = span_row.text
            tag = span_row.tag
            xminimo = span_row.xmin
            stile = span_row.span_font

            # if 'h' in tag or stile=='Helvetica-Bold' :
            if stile=='Helvetica-Bold' :

                if text in lista_keys_due_righe :
                    headings_list[-1] += ' ' + text
                    tmp = []
                    heading = text

                else:
                  if re.search(pattern, text):
                      headings_list.append('ID Client')
                      text_list.append('\n'.join(tmp))
                      tmp = []
                      heading = 'ID Client'
                      Id_cliente = re.search(pattern, text).group(1)
                      tmp.append(Id_cliente)
                  else:
                    headings_list.append(text)
                    text_list.append('\n'.join(tmp))
                    tmp = []
                    heading = text


            else:
                tmp.append(text)

        text_list.append('\n'.join(tmp))
        text_list = text_list[1:]
        text_df = pd.DataFrame(zip(headings_list, text_list), columns=['heading', 'content'])

        # data_dict = []
        # for heading, content in zip(headings_list, text_list):
        #     data_dict.append({ heading:content})

        # # return json.dumps(data_dict, indent=2)
        # return data_dict


        data_dict = {}
        data_dict['Cliente'] = 'Fincons'
        for heading, content in zip(headings_list, text_list):
            content = content.replace('\n', ' ')
            data_dict[heading] = content

        # Return the data as a dictionary
        return data_dict

    else:
      print('Questo file non è fincons')

      return {}

In [ ]:
def struttura_output(result_df):
  output = pd.DataFrame(result_df).T.reset_index().rename(columns={'index':'Nome File'})
  output['ID Client'] = output['ID Client'].astype(int)
  # Estrarre la data di inizio dalla colonna 'Tempo'
  output['Start'] = output['Data prevista inizio attivita\''].str.extract(r'(\d{2}-\d{2}-\d{4})')
  # Convertire la colonna 'start' in formato datetime
  output['Start'] = pd.to_datetime(output['Start'], format='%d-%m-%Y')
  try:
    # Estrarre la durata e l'unità dalla colonna 'Tempo'
    duration_info = output['Data prevista inizio attivita\''].str.extract(r'(\d+) (\w+)')

    # Mappare le unità temporali a giorni
    unit_to_days = {'giorno': 1, 'giorni': 1, 'mese': 30, 'mesi': 30, 'anno': 365, 'anni': 365}
    output['duration_days'] = duration_info[0].astype(int) * duration_info[1].map(unit_to_days)


    # Calcolare la data di fine aggiungendo la durata in giorni con relativedelta
    output['End date'] = output.apply(lambda row: row['Start'] + relativedelta(days=row['duration_days']), axis=1)

    # Eliminare le colonne temporanee se necessario
    output = output.drop(['duration_days'], axis=1)
  except:
    output['End date'] = ''

  return output

In [ ]:
# @title
folder_path = '/content/drive/MyDrive/progetto_CV_F2I/File_CV'
# output_file = '/content/drive/MyDrive/progetto_CV_F2I/output_json_03.json'
output = struttura_output(extract_pdf_folder(folder_path))
output

,Nome File,Cliente,ID Client,Richiedente,Stato,BU/Territorio beneficiario,Profilo,Sede di lavoro,Tipo impiego,Data prevista inizio attivita',...,Descrizione Sintetica Profilo,Job Description,Lingue obbligatorie,Skills obbligatorie,Lingue opzionali,Skills nice to have,Certificazioni,Note per il fornitore (es. necessita di pc o altro),Start,End date
0,RDA-9.pdf,Fincons,3345,Ricchi Federico,Pubblicata,BU Media,Developer - Area Web,Full Remote,FT - Full Time,13-12-2023 - durata prevista 12 mesi,...,Vanilla javaScript,Nell'ambito di un servizio esistente di manute...,Italiano (C1),- HTML5 (Alto) - Javascript (Eccellente),,,,NaN,2023-12-13,2024-12-07
1,RDA-10.pdf,Fincons,3144,Belviso Vito,Pubblicata,Territory Bari,DevOps - Area Web,Full Remote,FT - Full Time,02-07-2023 - durata prevista 12 mesi,...,System Engineer AWS,Systen engineer con almeno 4 anni di esperienz...,Italiano (C1),- AWS (Eccellente) - Bash scripting (Medio) - ...,Inglese (B1),- Jenkins (Medio) - Ansible (Medio),,Disponibile a trasferte spot a Bari,2023-07-02,2024-06-26
2,RDA-8.pdf,Fincons,3344,Ricchi Federico,Pubblicata,BU Media,Developer - Area Web,Full Remote - full remote ammesso,FT - Full Time,13-12-2023 - durata prevista 12 mesi,...,Front-End developer,Nell'ambito di un servizio esistente di manute...,Italiano (C1),- HTML5 (Eccellente) - Javascript (Alto),Inglese (B2),- React (Medio) - Angular (Medio),,NaN,2023-12-13,2024-12-07
3,RDA-7.pdf,Fincons,3314,Boccaperta Alessandra,Pubblicata,BU Financial Services,Developer - Area Web,Full Remote - Milano,FT - Full Time,23-11-2023 - durata prevista 12 mesi,...,Dev . Net,Risorsa da inserire nell' area IT-Danni per il...,Italiano (C1),- MS SQL (Alto) - Angular (Medio) - .Net/core ...,,- MVC (Medio) - Jquery (Medio) - MongoDB (Bass...,,NaN,2023-11-23,2024-11-17
4,RDA-6.pdf,Fincons,3339,Scapinello Emanuele,Pubblicata,BU Financial Services,Developer - Area Web,Full Remote - Italia,FT - Full Time,13-11-2023 - durata prevista 12 mesi,...,Sviluppatore senior PLSQL,Sviluppatore molto senior PLSQL in grado di cu...,Italiano (C2),- PlSql (Eccellente),,,,NaN,2023-11-13,2024-11-07
5,RDA-5.pdf,Fincons,3344,Ricchi Federico,Pubblicata,BU Media,Developer - Area Web,Full Remote - full remote ammesso,FT - Full Time,13-12-2023 - durata prevista 12 mesi,...,Front-End developer,Nell'ambito di un servizio esistente di manute...,Italiano (C1),- HTML5 (Eccellente) - Javascript (Alto),Inglese (B2),- React (Medio) - Angular (Medio),,NaN,2023-12-13,2024-12-07
6,RDA-4.pdf,Fincons,3345,Ricchi Federico,Pubblicata,BU Media,Developer - Area Web,Full Remote,FT - Full Time,13-12-2023 - durata prevista 12 mesi,...,Vanilla javaScript,Nell'ambito di un servizio esistente di manute...,Italiano (C1),- HTML5 (Alto) - Javascript (Eccellente),,,,NaN,2023-12-13,2024-12-07
7,RDA-3.pdf,Fincons,3321,Palombella Riccardo,Pubblicata,BU Financial Services,Technical Consultant - Area Web,Full Remote - Italia,FT - Full Time,04-12-2023 - durata prevista 6 mesi,...,Technical Business Analyst,The Technical Business Analyst (Insurance Indu...,Inglese (C1) Italiano (Madrelingua),- Azure (Alto) - Microservizi (Alto),,,,Full Remote erogato dall'Italia. Requirements:...,2023-12-04,2024-06-01
8,RDA-2.pdf,Fincons,3147,Marognoli Sonia,Pubblicata,BU Financial Services,Analyst - Area Web,Full Remote,FT - Full Time,18-09-2023 - durata prevista 12 mesi,...,BA funz/tecnico (banking),Il candidato sara inserito nel team Operation ...,Inglese (C1),- Altro (Medio),,- MySQL (Medio),,Potrebbe essere richiesta la presenza on site ...,2023-09-18,2024-09-12
9,RDA-1.pdf,Fincons,3293,Valerio Alessandro,Pubblicata,BU Financial Services,Functional Consultant - Area Web,Full Remote - Italia,PT - Part Time (50%),12-11-2023 - durata prevista 4 mesi,...,Data Governance specialist,Must have: Specialist in data governance and c...,Inglese (C1) Italiano (C1),- BI-Microsoft (Alto),,,,Ammesso full remote; richiesto part-time al 50...,2023-11-12,2024-03-11


In [ ]:
foglio = pd.read_excel('/content/drive/MyDrive/progetto_CV_F2I/Foglio-Fincons.xlsx')
foglio['ID Client'] = foglio['ID Client'].astype(int)
foglio

,Cliente,Categoria,Summary Description,ID Client,Tariffa,Job Description,Available Positions (Total),Start,End date,Location,DATA INSERIMENTO,Riferimento F2 per la richiesta,Manager Cliente,Stato richiesta
0,Fincons,Developer,Semior Developer .NET. - Area Web,3325,0,Senior Developer .NET. - Proven knowledge of M...,1,2023-11-30,2024-05-31,full remote,2023-11-02,Virginio,,A


In [ ]:
output.rename(columns={
    "Descrizione Sintetica Profilo": "Summary Description",
    "Profilo": "Categoria",
    "Job Description": "Job Description",
    "Ndeg Risorse richieste": "Available Positions (Total)",
    "Sede di lavoro": "Location"
}, inplace=True)

colonne_comuni = ['Cliente', 'Categoria', 'Summary Description', 'ID Client',
       'Job Description', 'Available Positions (Total)', 'Start', 'End date',
       'Location'] #foglio.columns.intersection(output.columns)


foglio_ = pd.concat([foglio, output[colonne_comuni]]).reset_index(drop=True)

In [ ]:
foglio_[foglio.columns]

,Cliente,Categoria,Summary Description,ID Client,Tariffa,Job Description,Available Positions (Total),Start,End date,Location,DATA INSERIMENTO,Riferimento F2 per la richiesta,Manager Cliente,Stato richiesta
0,Fincons,Developer,Semior Developer .NET. - Area Web,3325,0.0,Senior Developer .NET. - Proven knowledge of M...,1,2023-11-30,2024-05-31,full remote,2023-11-02,Virginio,,A
1,Fincons,Developer - Area Web,Vanilla javaScript,3345,NaN,Nell'ambito di un servizio esistente di manute...,1,2023-12-13,2024-12-07,Full Remote,NaT,NaN,NaN,NaN
2,Fincons,DevOps - Area Web,System Engineer AWS,3144,NaN,Systen engineer con almeno 4 anni di esperienz...,1,2023-07-02,2024-06-26,Full Remote,NaT,NaN,NaN,NaN
3,Fincons,Developer - Area Web,Front-End developer,3344,NaN,Nell'ambito di un servizio esistente di manute...,1,2023-12-13,2024-12-07,Full Remote - full remote ammesso,NaT,NaN,NaN,NaN
4,Fincons,Developer - Area Web,Dev . Net,3314,NaN,Risorsa da inserire nell' area IT-Danni per il...,1,2023-11-23,2024-11-17,Full Remote - Milano,NaT,NaN,NaN,NaN
5,Fincons,Developer - Area Web,Sviluppatore senior PLSQL,3339,NaN,Sviluppatore molto senior PLSQL in grado di cu...,1,2023-11-13,2024-11-07,Full Remote - Italia,NaT,NaN,NaN,NaN
6,Fincons,Developer - Area Web,Front-End developer,3344,NaN,Nell'ambito di un servizio esistente di manute...,1,2023-12-13,2024-12-07,Full Remote - full remote ammesso,NaT,NaN,NaN,NaN
7,Fincons,Developer - Area Web,Vanilla javaScript,3345,NaN,Nell'ambito di un servizio esistente di manute...,1,2023-12-13,2024-12-07,Full Remote,NaT,NaN,NaN,NaN
8,Fincons,Technical Consultant - Area Web,Technical Business Analyst,3321,NaN,The Technical Business Analyst (Insurance Indu...,1,2023-12-04,2024-06-01,Full Remote - Italia,NaT,NaN,NaN,NaN
9,Fincons,Analyst - Area Web,BA funz/tecnico (banking),3147,NaN,Il candidato sara inserito nel team Operation ...,1,2023-09-18,2024-09-12,Full Remote,NaT,NaN,NaN,NaN


In [ ]:
foglio_[foglio.columns].to_excel('/content/drive/MyDrive/progetto_CV_F2I/Foglio-Fincons_aggiornato.xlsx')